In [ ]:
! pip install jsonlines

In [ ]:
import pandas as pd
import numpy as np
import os
import jsonlines
import re
import matplotlib.pyplot as plt
import seaborn as sns
import warnings 
warnings.filterwarnings('ignore')
from tqdm import tqdm
from nltk.corpus import stopwords
from wordcloud import WordCloud
import nltk
from nltk.tokenize import word_tokenize
from collections import Counter

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
'''THIS CELL READS THE LANG-8 DATASET FILE AND EXTRACTS INCORRECT AND CORRECT SENTENCES ONLY'''

f1 = open("/content/drive/MyDrive/Colab Notebooks/cs2/lang-8/lang-8-en-1.0/entries.train") # OPENING THE FILE 
lines1 = f1.readlines() # READING THE LINES
inp1 = [] # LIST FOR STORING INCORRECT SENTENCES
tgt1 = [] # LIST FOR STORING

for i in lines1: # FOR EACH LINE 
    lst = i.split("\t") # WE ARE SPLITTING THE LINE AT \t
    
# IF LENGTH OF THE LIST IS GREATER THAN 5 THEN CORRECT SENTTENCE EXISTS OTHERWISE ONLY INCORRECT SENTENCE IS PRESENT 
    if len(lst)>5  :     #IF LENGTH IS GREATER THAN 5  
        inp1.append(lst[-2]) # APPEND SECONG LAST ITEM IN LIST WHICH IS INCORRECT SENTENCE
        tgt1.append(lst[-1]) # APPEND LAST ITEM IN THE LIST WHICH IS CORRECT SENTECE.

In [ ]:
'''THIS CELL READS THE SMS_TEXT DATASET FILE AND EXTRACTS INCORRECT AND CORRECT SENTENCES ONLY'''


f2 = open("/content/drive/MyDrive/Colab Notebooks/cs2/text_sms/release/en2cn-2k.en2nen2cn","r",encoding="UTF-8") # READING THE FILE

lines2 = f2.readlines() # STORING ALL THE LINES IN A VARIABLE
inp2 = [] # LIST FOR STORING INCORRECT SENTENCES
tgt2 = [] # LIST FOR STORING CORRECT SENTENCES

# THE DASET CONTAINS 2000 DATAPOINTS, THEREFORE RUNNING THE LOOP FOR 2000 TIMES
for i in range(2000): 
    inp2.append(lines2[i*3]) #APPEDING FIRST ROW FOR EACH DATAPOINT
    tgt2.append(lines2[i*3+1]) # APPENDING SECOND ROW FOR EACH DATAPOINT

## Combining datasets

In [ ]:
'''THIS CELL COMBINES BOTH THE DATASETS TOGETHER'''

df = pd.DataFrame() # CREATING THE DATAFRAME
df["input"] = inp1+inp2 # ADDING BOTH THE LISTS OF INPUTS TO ONE COLUMN
df["output"] =  tgt1+tgt2 # ADDING BOTH THE LISTS OF TARGETS TO ONE COLUMN
df["y"] = list("1"*len(inp1)) + list("2"*len(inp2))

### Data Preprocessing 

In [ ]:
'''THIS FUNCTION REMOVES THE SPACES BETWEEN THE CONTRACTED WORDS AND REMOVING UNNECESSARY SPACES IN THE SENTENCES
            ca n't ==> can't 
            I 'm ===> I'm ...etc
'''
def remove_spaces(text):
    text = re.sub(r" '(\w)",r"'\1",text)
    text = re.sub(r" \,",",",text)
    text = re.sub(r" \.+",".",text)
    text = re.sub(r" \!+","!",text)
    text = re.sub(r" \?+","?",text)
    text = re.sub(" n't","n't",text)
    text = re.sub("[\(\)\;\_\^\`\/]","",text)
    
    return text


'''THIS FUNCTION DECONTRACTS THE CONTRACTED WORDS'''
#REF : https://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python

def decontract(text):
    text = re.sub(r"won\'t", "will not", text)
    text = re.sub(r"can\'t", "can not", text)
    text = re.sub(r"n\'t", " not", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'s", " is", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'t", " not", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'m", " am", text)
    return text


'''THIS FUNCTION PREPROCESSES THE TEXT '''
def preprocess(text):
    text = re.sub("\n","",text)
    text = remove_spaces(text)   # REMOVING UNWANTED SPACES
    text = re.sub(r"\.+",".",text)
    text = re.sub(r"\!+","!",text)
    text = decontract(text)    # DECONTRACTION
    text = re.sub("[^A-Za-z0-9 ]+","",text)
    text = text.lower()
    return text

In [ ]:
'''HERE WE ARE APPLYIN PREPROCESS FUNCTION TO INPUT AND OUTPUT SENTENCES'''

df["processed_input"] = df.input.apply(preprocess) # APPLYING PREPROCESS FUNCTION TO INPUT 
df["processed_output"] = df.output.apply(preprocess) # APPLYING PREPROCESS FUNCION TO OUTPUT

In [ ]:
df =df.drop(["input","output"],axis=1)

## Exploartory Data Analysis

### Removing null values

In [ ]:
'''THIS CELL REMOVES ROWS WITH NULL VALUES IN INPUT AND OUTPUT TEXT'''

df = df[df.processed_input.notnull()]
df = df[df.processed_output.notnull()]

#### Observations:
* By removing the null values we have reduced the number of rows from 511163 to 511123

### Removing Duplicates

In [ ]:
'''DROPPING THE DUPLICATES'''

df = df.drop_duplicates()

#### OBSERVATION:
* By dropping duplicates we can reduce the number of datapoints from 511123 to 505890

# Wrong words and Corrected words analysis

#####  Wrong words : words present in input but not present in target sentences 
##### Corrected words : words present in target sentences but not present in input sentences.

In [ ]:
'''THIS FUNCTION TAKES IN THE DATAFRAME AND OUPUTS THE CORPUS FOR WRONG WORDS AND CORRECTED WORDS'''

def wrong_words(df):
    # CONVERTING THE SENTENCES OF INPUT TO WORDS AND CONVERTING TO SETS
    inp = df.processed_input.apply(lambda x: set(str(x).split()))
    
    # CONVERTING THE SENTENCES OF OUTPUT TO WORDS AND CONVERTING TO SETS
    out = df.processed_output.apply(lambda x: set(str(x).split()))
    
    # GETTIN THE SET OF WRONG WORDS 
    df_ww = (inp-out).apply(list)
    
    # GETTING THE ST OF CORRECTED SENTENCES
    df_corrected = (out-inp).apply(list)
    
    # JOINING ALL WORDS AND FORIMING A LONG TEXT 
    ww_corpus = ""
    for i in df_ww.values:
        
        if len(i)>0:
            ww_corpus+= " "+ " ".join(i) 
            
    # JOINING ALL WORDS AND FORIMING A LONG TEXT 
    corr_corpus =""       
    for i in df_corrected.values:
        if len(i)>0:
            corr_corpus += " "+ " ".join(i)    
        
    # RETURNING THE WRONG WORDS TEXT AND CORRECTED WORDS TEXT          
    return ww_corpus,corr_corpus

In [ ]:
'''THIS CELL FORMS THE WORD CLOUD FOR WRONG WORDS AND CORRECTED WORDS'''

# GETTING THE TEXT FOR WRONG WORDS AND CORRECTED WORDS
ww_corpus,corr_corpus = wrong_words(df)

# FORMING WORD CLOUD FOR WROND WORDS
wordcloud_ww = WordCloud(width = 800, height = 800,
                 background_color ='black',
                 min_font_size = 10).generate(ww_corpus)

# FORMING WORD CLOUD FOR CORRECTED WORDS
wordcloud_corr = WordCloud(width = 800, height = 800,
                 background_color ='black',
                 min_font_size = 10).generate(corr_corpus)



## POS Tagging

# Input

In [ ]:
def pos(text):
    pos = nltk.pos_tag(word_tokenize(text))
    pos = [j for i ,j in pos]
    return pos

tags_input = df.processed_input.apply(lambda x: pos(x))
tags_input = [ j for i in tags_input for j in i]
counter_input = Counter(tags_input) 
inp_10 = list(zip(*counter_input.most_common(10)))

# Output

In [ ]:
def pos(text):
    pos = nltk.pos_tag(word_tokenize(text))
    pos = [j for i ,j in pos]
    return pos

tags_output = df.processed_output.apply(lambda x: pos(x))
tags_output = [ j for i in tags_output for j in i]
counter_output = Counter(tags_output) 
out_10 = list(zip(*counter_output.most_common(10)))

In [ ]:
df[["processed_input","processed_output","y"]].to_csv("/content/drive/MyDrive/Colab Notebooks/cs2/processed_data.csv",index=False)